In [1]:
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy swifter
!pip install flow_vis

     |████████████████████████████████| 647 kB 2.9 MB/s 
     |████████████████████████████████| 281 kB 90.7 MB/s 
     |████████████████████████████████| 140 kB 55.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#import torch
import os
import glob
import numpy as np
import random
import pandas as pd
from time import sleep
import cv2 
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
import mediapy as media
from google.colab.patches import cv2_imshow
import mediapy as media
import flow_vis

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Optical Flow

In [4]:
def flow_to_color(flow, hsv):
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    hsv[..., 0] = ang*180/np.pi/2
    hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

def getOpticalFlow(video, return_rgb_flow=True):
    """Calculate dense optical flow of input video
    Args:
        video: the input video with shape of [frames,height,width,channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames,height,width,channel]
        flows_y: the optical flow at y-axis, with the shape of [frames,height,width,channel]
    """
    # initialize the list of optical flows
    
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img,(224,224,1)))

    flows = []
    rgb_flows = []
    for i in range(0,len(video)-1):
        # calculate optical flow between each pair of frames


        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i+1], None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0],None,0,255,cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1],None,0,255,cv2.NORM_MINMAX)
        # Add into list 

        gss_app = cv2.GaussianBlur(flow,(5,5),1)

        if return_rgb_flow:
          flow_color = flow_vis.flow_to_color(gss_app, convert_to_bgr=False)
          rgb_flows.append(flow_color)

        flows.append(gss_app)
        
    # Padding the last frame as empty array
    flows.append(np.zeros((224,224,2)))

    flows = normalize(flows)
      
    return (np.array(flows, dtype=np.float32), np.array(rgb_flows))

Sliding windows to find most movimented area

## Global Variables

In [5]:
# Paths base
PATH_BASE = '/content/drive/MyDrive/ucf_experiments/'
PATH_CICLE = PATH_BASE + 'ciclo_experimental_1/'

# Path base de armazenamento dos augmentations
PATH_DATA = PATH_CICLE + 'data/'

# Dados de treino augmentados
PATH_AUG_BASE = PATH_DATA + 'augmented_train/'
# Dados de teste redimensionados (visto que não popdem ser augmentados)
PATH_TEST_RESIZED = PATH_DATA + 'resized_test/'

In [6]:
# Create target directory & all intermediate directories if don't exists
def create_dir(dirName):
  try:
      os.makedirs(dirName)    
      print("Directory " , dirName ,  " Created ")
  except FileExistsError:
      print("Directory " , dirName ,  " already exists")  

## Data Processing

In [7]:
videos = pd.read_csv(PATH_AUG_BASE + "train_for_network.txt", sep=' ', names=['path', 'label'])
videos_test = pd.read_csv(PATH_TEST_RESIZED + "x_test_resized.csv")

In [8]:
videos_test = videos_test[['resized', 'label']].rename(columns={'resized': 'path'})

In [9]:
videos_test

,path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
...,...,...
1052,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1053,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1054,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1055,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


## Test Process Function

In [12]:
def normalize(data):
    mean = np.mean(data)
    std = np.std(data)
    return (data-mean) / std

def dynamic_crop(opt_flows, rgb_frames):

    rgb_video = rgb_frames.copy()
    # sum of optical flow magnitude of individual frame
    magnitude = np.sum(opt_flows, axis=0)

    print('Magnitude: ', str(magnitude.shape))
    # filter slight noise by threshold 
    thresh = np.mean(magnitude)
    magnitude[magnitude<thresh] = 0
    # calculate center of gravity of magnitude map and adding 0.001 to avoid empty value
    x_pdf = np.sum(magnitude, axis=1) + 0.001
    y_pdf = np.sum(magnitude, axis=0) + 0.001
    # normalize PDF of x and y so that the sum of probs = 1
    x_pdf /= np.sum(x_pdf)
    y_pdf /= np.sum(y_pdf)

    x_pdf = x_pdf.sum(axis=1)
    y_pdf = y_pdf.sum(axis=1)

    # randomly choose some candidates for x and y 
    x_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)
    y_points = np.random.choice(a=np.arange(224), size=10, replace=True, p=y_pdf)
    # get the mean of x and y coordinates for better robustness
    x = int(np.mean(x_points))
    y = int(np.mean(y_points))
    # avoid to beyond boundaries of array
    x = max(56,min(x,167))
    y = max(56,min(y,167))
    # get cropped video

    cropped = rgb_video[:,x-56:x+56,y-56:y+56,:].copy()

    w = 112
    h = 112

    start_point = (int(x-w/2), int(y-h/2))
    end_point = (int(x+w/2), int(y+h/2))

    color = (255,255,255)
    thickness = -1
    blurred_video = np.zeros_like(rgb_video)

    for idx, frame in enumerate(rgb_video):
      blurred_img = cv2.GaussianBlur(frame, (21, 21), 0)
      mask = np.zeros((frame.shape[0], frame.shape[1], 3), dtype=np.uint8)
      mask = cv2.rectangle(mask, start_point, end_point, color, thickness)

      best_cut = np.where(mask==np.array([255, 255, 255]), frame, blurred_img)
      blurred_video[idx] = best_cut

    return cropped, blurred_video

### Teste Visual

In [13]:
sample_url = videos_test.sample().path.squeeze()
sample_video = media.read_video(sample_url)

opt_frames, rgb_opt_frames = getOpticalFlow(sample_video, return_rgb_flow=True)

sample_url

#/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/resized_test/nVdn1krnTCQ_3-12aaa3e64a_1_resized.mp4 blur diferente do crop
#/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/resized_test/VF8M3gXp_0-5ea7c1e138_0_resized.mp4

'/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/resized_test/Arrest014_x264-69ddbe8918_0_resized.mp4'

In [16]:
opt_frames.shape

(48, 224, 224, 2)

In [21]:
magnitude = opt_frames.sum(axis=0)

In [31]:
thresh = np.mean(magnitude)
magnitude[magnitude<thresh] = 0

In [32]:
x_pdf = np.sum(magnitude, axis=1) + 0.001
y_pdf = np.sum(magnitude, axis=0) + 0.001

In [33]:
x_pdf.shape

(224, 2)

In [34]:
x_pdf /= np.sum(x_pdf)
y_pdf /= np.sum(y_pdf)

In [35]:
x_pdf.shape

(224, 2)

In [36]:
x_pdf = x_pdf.sum(axis=1)
y_pdf = y_pdf.sum(axis=1)

In [37]:
x_pdf.shape

(224,)

In [38]:
np.random.choice(a=np.arange(224), size=10, replace=True, p=x_pdf)

array([ 66, 193, 115, 140, 156, 166,  75, 223,  27, 149])

In [27]:
x_pdf.shape

(224, 2)

In [14]:
rgb_cut, rgb_blur = dynamic_crop(opt_frames, sample_video)

Magnitude:  (224, 224, 2)


In [ ]:
media.show_videos([sample_video, rgb_opt_frames, rgb_blur, rgb_cut])

This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.,This browser does not support the video tag.


### Test Yolo

In [ ]:
"""video_ext_result = []

for rgb_frame in rgb_cut:
  pred = model(rgb_frame)
  detection_df  = pred.pandas().xyxy[0]
  has_person    = 'person' in detection_df.name.unique()
  if has_person:
    video_ext_result.append(rgb_frame[0])

len(video_ext_result)"""

"video_ext_result = []\n\nfor rgb_frame in rgb_cut:\n  pred = model(rgb_frame)\n  detection_df  = pred.pandas().xyxy[0]\n  has_person    = 'person' in detection_df.name.unique()\n  if has_person:\n    video_ext_result.append(rgb_frame[0])\n\nlen(video_ext_result)"

## Spliting

In [ ]:
splitted = np.array_split(videos, 200)
splitted_test = np.array_split(videos_test, 10)

In [ ]:
def process_videos(videos_df, save_blur_base, save_crop_base, only_humans=True):
  videos = []

  for idx, row in videos_df.iterrows():
    video_path = row['path']
    label = row['label']

    rgb_frames = media.read_video(video_path)
    opt_frames, rgb_opt_frames = getOpticalFlow(rgb_frames, return_rgb_flow=False)
    result_crop_frames = []
    result_blur_frames = []


    rgb_cut, rgb_blur = dynamic_crop(opt_frames, rgb_frames)

    if only_humans: 
      for rgb_frame, rgb_blur_frame in zip(rgb_cut, rgb_blur):
        pred = model(rgb_frame)
        detection_df  = pred.pandas().xyxy[0]
        has_person    = 'person' in detection_df.name.unique()
        if has_person:
          result_crop_frames.append(rgb_frame)
          result_blur_frames.append(rgb_blur_frame)
    else:
      result_crop_frames = rgb_cut
      result_blur_frames = rgb_blur

    if len(result_crop_frames) < 1:
      continue

    video_blur_name = video_path.split('/')[-1].split('.')[0] + '_blur.mp4'
    video_crop_name = video_path.split('/')[-1].split('.')[0] + '_crop.mp4'

    video_blur_output_path = os.path.join(save_blur_base, video_blur_name)
    video_crop_output_path = os.path.join(save_crop_base, video_crop_name)

    media.write_video(video_blur_output_path, np.array(result_blur_frames))
    media.write_video(video_crop_output_path, np.array(result_crop_frames))

    row = {
      'crop_path': video_crop_output_path,
      'blur_path': video_blur_output_path,
      'label': label
    }

    videos.append((video_blur_output_path, video_crop_output_path, label))

  return videos

In [ ]:
#glob.glob('/content/drive/MyDrive/ucf_experiments/experiment_3/data_checkpoints/train_nony_checkpoints/*_checkpoin.csv')[-1:]

In [ ]:
len(splitted)

200

## Train Data

In [ ]:
PATH_CROP = PATH_DATA + 'data_crop/'
PATH_BLUR = PATH_DATA + 'data_blur/'

PATH_CROP_TRAIN = PATH_CROP + 'train/'
PATH_CROP_TEST = PATH_CROP + 'test/'

PATH_BLUR_TRAIN = PATH_BLUR + 'train/'
PATH_BLUR_TEST = PATH_BLUR + 'test/'

PATH_CHECKPOINTS = PATH_DATA + 'data_checkpoints/'
PATH_CHECKPOINTS_TRAIN = PATH_CHECKPOINTS + 'train/'
PATH_CHECKPOINTS_TEST = PATH_CHECKPOINTS + 'test/'

create_dir(PATH_CROP)
create_dir(PATH_BLUR)
create_dir(PATH_CROP_TRAIN)
create_dir(PATH_CROP_TEST)
create_dir(PATH_BLUR_TRAIN)
create_dir(PATH_BLUR_TEST)

create_dir(PATH_CHECKPOINTS)
create_dir(PATH_CHECKPOINTS_TRAIN)
create_dir(PATH_CHECKPOINTS_TEST)

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/test/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/train/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/test/  already exists


In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 195

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=(len(splitted) - start_idx)):
  
  results = process_videos(
      split,
      save_blur_base=PATH_BLUR_TRAIN,
      save_crop_base=PATH_CROP_TRAIN,
      only_humans=False
  )

  checkpoints = checkpoints.append(results, ignore_index=True)
  checkpoints.to_csv(PATH_CHECKPOINTS_TRAIN+'{}_checkpoin.csv'.format(str(start_idx + idx)))
  sleep(5)

NameError: ignored

## Test Data

In [ ]:
checkpoints = pd.DataFrame([])
start_idx = 0

for idx, split in tqdm(enumerate(splitted_test[start_idx:]), total=len(splitted_test) - start_idx):
  
  results = process_videos(
      split,
      save_blur_base=PATH_BLUR_TEST,
      save_crop_base=PATH_CROP_TEST,
      only_humans=False
  )

  checkpoints = checkpoints.append(results, ignore_index=True)
  checkpoints.to_csv(PATH_CHECKPOINTS_TEST+'{}_checkpoin.csv'.format(str(start_idx + idx)))
  sleep(5)

100%|██████████| 10/10 [1:17:33<00:00, 465.34s/it]


In [ ]:
"""sample = videos.sample()
cuts = process_videos(sample)

original_video = media.read_video(sample.path.squeeze())
path, cut = cuts[0]

if len(cut) > 0:
  media.show_videos([original_video, cut])
else:
  print('No one detected')"""

"sample = videos.sample()\ncuts = process_videos(sample)\n\noriginal_video = media.read_video(sample.path.squeeze())\npath, cut = cuts[0]\n\nif len(cut) > 0:\n  media.show_videos([original_video, cut])\nelse:\n  print('No one detected')"

In [ ]:
"""optical, rgb_optical = getOpticalFlow(original_video, return_rgb_flow=True)
media.show_videos([original_video, rgb_optical, cut])"""

'optical, rgb_optical = getOpticalFlow(original_video, return_rgb_flow=True)\nmedia.show_videos([original_video, rgb_optical, cut])'

In [ ]:
PATH_CHECKPOINTS_TRAIN #tem barra? Se sim, só siga

'/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/train/'

In [ ]:
train_parts = glob.glob(PATH_CHECKPOINTS_TRAIN  +'*_checkpoin.csv')
test_parts = glob.glob(PATH_CHECKPOINTS_TEST + '*_checkpoin.csv')

In [ ]:
print(len(train_parts))
print(len(test_parts))

200
10


In [ ]:
files_train = pd.concat([ pd.read_csv(path) for path in train_parts ])
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])

In [ ]:
files_train

,Unnamed: 0,0,1,2
0,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
2,2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
3,3,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
4,4,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
...,...,...,...,...
520,520,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
521,521,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
522,522,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
523,523,/content/drive/MyDrive/ucf_experiments/ciclo_e...,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


In [ ]:
def processed_data(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_blur = processed_data[['0', '2']]
  dynamic_crop = processed_data[['1', '2']]

  return dynamic_blur, dynamic_crop

In [ ]:
dynamic_blur_train, dynamic_crop_train = processed_data(files_train)
print(dynamic_blur_train.shape)
print(dynamic_crop_train.shape)

(21135, 2)
(21135, 2)


In [ ]:
dynamic_blur_train.head(2)['0'].to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train/Robbery097_x264-c083f63c77_0_resized_blur.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/train/Robbery035_x264-a4f337e6cb_1_resized_blur.mp4']

In [ ]:
dynamic_crop_train.head(2)['1'].to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/Robbery097_x264-c083f63c77_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/Robbery035_x264-a4f337e6cb_1_resized_crop.mp4']

In [ ]:
dynamic_blur_test, dynamic_crop_test = processed_data(files_test)
print(dynamic_blur_test.shape)
print(dynamic_crop_test.shape)

(1057, 2)
(1057, 2)


In [ ]:
dynamic_blur_test.head(2)['0'].to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/test/Fighting051_x264-e2c36a76ea_1_resized_blur.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_blur/test/Assault014_x264-ebcf10cc63_0_resized_blur.mp4']

In [ ]:
dynamic_crop_test.head(2)['1'].to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Fighting051_x264-e2c36a76ea_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Assault014_x264-ebcf10cc63_0_resized_crop.mp4']

In [ ]:
dynamic_blur_train.to_csv(PATH_BLUR + 'train.txt', header=None, index=False, sep=';')
dynamic_blur_test.to_csv(PATH_BLUR + 'test.txt', header=None, index=False, sep=';')

In [ ]:
dynamic_crop_train.to_csv(PATH_CROP + 'train.txt', header=None, index=False, sep=';')
dynamic_crop_test.to_csv(PATH_CROP + 'test.txt', header=None, index=False, sep=';')

# Redimensionamento (Crop - Train)

In [ ]:
# Paths base
PATH_BASE = '/content/drive/MyDrive/ucf_experiments/'
PATH_CICLE = PATH_BASE + 'ciclo_experimental_1/'

# Path base de armazenamento dos augmentations
PATH_DATA = PATH_CICLE + 'data/'
PATH_CROP = PATH_DATA + 'data_crop/'
PATH_CROP_TRAIN = PATH_CROP + 'train/'
PATH_CROP_TRAIN_RESIZED = PATH_CROP + 'train_resized/'
PATH_CROP_TEST_RESIZED = PATH_CROP + 'test_resized/'

PATH_CHECKPOINTS = PATH_DATA + 'data_checkpoints/'
PATH_CHECKPOINTS_TRAIN_RESIZED = PATH_CHECKPOINTS + 'train_resized/'
PATH_CHECKPOINTS_TEST_RESIZED = PATH_CHECKPOINTS + 'test_resized/'

create_dir(PATH_CROP_TRAIN_RESIZED)
create_dir(PATH_CROP_TEST_RESIZED)
create_dir(PATH_CHECKPOINTS)
create_dir(PATH_CHECKPOINTS_TRAIN_RESIZED)
create_dir(PATH_CHECKPOINTS_TEST_RESIZED)

Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train_resized/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test_resized/  Created 
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/train_resized/  already exists
Directory  /content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_checkpoints/test_resized/  Created 


In [ ]:
# Load
videos_treino = pd.read_csv(PATH_CROP + 'train.txt', names=['path', 'label'], sep=';')
splitted = np.array_split(videos_treino, 100)

In [ ]:
videos_treino.shape

(21135, 2)

In [ ]:
videos_treino.head(2)

,path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1


In [ ]:
videos_treino.head(3).path.to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/Robbery097_x264-c083f63c77_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/Robbery035_x264-a4f337e6cb_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/train/Fighting028_x264-6b5602eb75_1_resized_crop.mp4']

In [ ]:
import swifter

def resized_videos(row):
  videos = []

  video_path = row['path']
  new_path = PATH_CROP_TRAIN_RESIZED + row['path'].split('/')[-1]

  rgb_frames = media.read_video(video_path)
  
  rgb_frames = media.to_float01(rgb_frames)

  sampled_video = tf.image.resize(rgb_frames, [224,224]).numpy()
  media.write_video(new_path, sampled_video)
  """row['asdasdwd'] = adsad
  row['asdasdwd'] = adsad
  row['asdasdwd'] = adsad
  row['asdasdwd'] = adsad"""


  return new_path


checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.swifter.apply(resized_videos, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CHECKPOINTS_TRAIN_RESIZED+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

  0%|          | 0/100 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/212 [00:00<?, ?it/s]

  1%|          | 1/100 [03:57<6:31:26, 237.24s/it]

Pandas Apply:   0%|          | 0/212 [00:00<?, ?it/s]

  2%|▏         | 2/100 [07:11<5:45:54, 211.78s/it]

...

 98%|█████████▊| 98/100 [6:08:04<08:12, 246.12s/it]

Pandas Apply:   0%|          | 0/211 [00:00<?, ?it/s]

 99%|█████████▉| 99/100 [6:12:18<04:08, 248.44s/it]

Pandas Apply:   0%|          | 0/211 [00:00<?, ?it/s]

100%|██████████| 100/100 [6:16:33<00:00, 225.93s/it]


In [ ]:
train_parts = glob.glob(PATH_CHECKPOINTS_TRAIN_RESIZED  +'*_checkpoint.csv')
print(len(train_parts))

100


In [ ]:
files_train = pd.concat([ pd.read_csv(path) for path in train_parts ])
files_train.shape

(21135, 4)

In [ ]:
files_train.head(3)

,Unnamed: 0,path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...
1,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...
2,2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...


In [ ]:
def processed_data_resized(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_crop = processed_data[['new_path', 'label']]

  return dynamic_crop

In [ ]:
dynamic_crop_train_resized = processed_data_resized(files_train)
print(dynamic_crop_train_resized.shape)

(21135, 2)


In [ ]:
PATH_CROP

'/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/'

In [ ]:
dynamic_crop_train_resized[['new_path', 'label']].to_csv(PATH_CROP + 'train_resized.txt', header=None, index=False, sep=';')

# Redimensionamento Crop - Teste


In [ ]:
# Load
videos_teste = pd.read_csv(PATH_CROP + 'test.txt', names=['path', 'label'], sep=';')
splitted = np.array_split(videos_teste, 10)

In [ ]:
videos_teste.shape

(1057, 2)

In [ ]:
videos_teste.head(2)

,path,label
0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1
1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0


In [ ]:
videos_teste.head(3).path.to_list()

['/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Fighting051_x264-e2c36a76ea_1_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Assault014_x264-ebcf10cc63_0_resized_crop.mp4',
 '/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/test/Fighting002_x264-8104c4ee2c_0_resized_crop.mp4']

In [ ]:
import swifter

def resized_videos(row):
  videos = []

  video_path = row['path']
  new_path = PATH_CROP_TEST_RESIZED + row['path'].split('/')[-1]

  rgb_frames = media.read_video(video_path)
  
  rgb_frames = media.to_float01(rgb_frames)

  sampled_video = tf.image.resize(rgb_frames, [224,224]).numpy()
  media.write_video(new_path, sampled_video)

  return new_path


checkpoints = pd.DataFrame([])
start_idx = 0
results = pd.DataFrame()

for idx, split in tqdm(enumerate(splitted[start_idx:]), total=len(splitted) - start_idx):
  split['new_path'] = split.swifter.apply(resized_videos, axis=1)

  # results.append(result, ignore_index=True)
  # checkpoints = checkpoints.append(results, ignore_index=True)
  split.to_csv(PATH_CHECKPOINTS_TEST_RESIZED+'{}_checkpoint.csv'.format(str(start_idx + idx)))
  sleep(5)

  0%|          | 0/10 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/106 [00:00<?, ?it/s]

 10%|█         | 1/10 [02:41<24:15, 161.75s/it]

Pandas Apply:   0%|          | 0/106 [00:00<?, ?it/s]

 20%|██        | 2/10 [05:30<22:04, 165.58s/it]

...

 80%|████████  | 8/10 [22:13<05:39, 169.55s/it]

Pandas Apply:   0%|          | 0/105 [00:00<?, ?it/s]

 90%|█████████ | 9/10 [25:02<02:49, 169.50s/it]

Pandas Apply:   0%|          | 0/105 [00:00<?, ?it/s]

100%|██████████| 10/10 [27:48<00:00, 166.80s/it]


In [ ]:
test_parts = glob.glob(PATH_CHECKPOINTS_TEST_RESIZED  +'*_checkpoint.csv')
print(len(test_parts))

10


In [ ]:
files_test = pd.concat([ pd.read_csv(path) for path in test_parts ])
files_test.shape

(1057, 4)

In [ ]:
files_test.head(3)

,Unnamed: 0,path,label,new_path
0,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...
1,1,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...
2,2,/content/drive/MyDrive/ucf_experiments/ciclo_e...,0,/content/drive/MyDrive/ucf_experiments/ciclo_e...


In [ ]:
def processed_data_resized(data_files):
  processed_data = data_files.drop_duplicates()
  processed_data = processed_data.drop(columns=['Unnamed: 0'])

  processed_data = processed_data.dropna()
  processed_data = processed_data.reset_index(drop=True)

  dynamic_crop = processed_data[['new_path', 'label']]

  return dynamic_crop

In [ ]:
dynamic_crop_test_resized = processed_data_resized(files_test)
print(dynamic_crop_test_resized.shape)

(1057, 2)


In [ ]:
PATH_CROP

'/content/drive/MyDrive/ucf_experiments/ciclo_experimental_1/data/data_crop/'

In [ ]:
dynamic_crop_test_resized[['new_path', 'label']].to_csv(PATH_CROP + 'test_resized.txt', header=None, index=False, sep=';')